# Train YOLOv8n-cls cho Chicken Disease Classification

Sử dụng train_aug.csv đã được augment


In [2]:
# Cài đặt ultralytics (YOLOv8)
%pip install ultralytics -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.1 MB/s eta 0:00:00


In [3]:
# Import thư viện
import os
import shutil
from pathlib import Path
import pandas as pd
from ultralytics import YOLO
from google.colab import drive
import torch

print("Đã import xong!")

# KIỂM TRA GPU
print("\n" + "="*50)
print("KIỂM TRA GPU")
print("="*50)
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"Current device: {torch.cuda.current_device()}")
else:
    print("⚠ KHÔNG CÓ GPU! Vui lòng:")
    print("  1. Vào Runtime → Change runtime type")
    print("  2. Chọn Hardware accelerator: GPU")
    print("  3. Runtime type: Python 3")
    print("  4. Nhấn Save và restart runtime")
print("="*50)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Đã import xong!

KIỂM TRA GPU
CUDA available: False
⚠ KHÔNG CÓ GPU! Vui lòng:
  1. Vào Runtime → Change runtime type
  2. Chọn Hardware accelerator: GPU
  3. Runtime type: Python 3
  4. Nhấn Save và restart runtime


## Bước 1: Mount Google Drive (nếu dữ liệu ở Drive)


In [4]:
# Mount Google Drive (bỏ comment nếu dữ liệu ở Drive)
drive.mount('/content/drive')

# Hoặc upload dữ liệu trực tiếp lên Colab
# Sử dụng tab Files bên trái để upload thư mục data/data_phan_chicken


Mounted at /content/drive


## Bước 2: Chuẩn bị dữ liệu theo format YOLO


In [5]:
# Đường dẫn dữ liệu (điều chỉnh theo vị trí dữ liệu của bạn)
# Nếu upload lên Colab: /content/data/data_phan_chicken
# Nếu từ Drive: /content/drive/MyDrive/AI_Chicken/data/data_phan_chicken

DATA_ROOT = Path('/content/drive/MyDrive/AI_Chicken/data/data_phan_chicken')
IMAGES_DIR = DATA_ROOT / 'data'
TRAIN_CSV = DATA_ROOT / 'train_aug.csv'
VAL_CSV = DATA_ROOT / 'val.csv'

# Tạo thư mục dataset theo format YOLO
YOLO_DATASET = Path('/content/drive/MyDrive/AI_Chicken/yolo_dataset')
YOLO_DATASET.mkdir(exist_ok=True)

TRAIN_DIR = YOLO_DATASET / 'train'
VAL_DIR = YOLO_DATASET / 'val'
TRAIN_DIR.mkdir(exist_ok=True)
VAL_DIR.mkdir(exist_ok=True)

print(f"Images dir: {IMAGES_DIR}")
print(f"Train CSV: {TRAIN_CSV}")
print(f"Val CSV: {VAL_CSV}")


Images dir: /content/drive/MyDrive/AI_Chicken/data/data_phan_chicken/data
Train CSV: /content/drive/MyDrive/AI_Chicken/data/data_phan_chicken/train_aug.csv
Val CSV: /content/drive/MyDrive/AI_Chicken/data/data_phan_chicken/val.csv


In [ ]:
# Đọc CSV và tạo thư mục theo class
train_df = pd.read_csv(TRAIN_CSV)
val_df = pd.read_csv(VAL_CSV)

print(f"Train samples: {len(train_df)}")
print(f"Val samples: {len(val_df)}")
print(f"\nTrain distribution:")
print(train_df['label'].value_counts())
print(f"\nVal distribution:")
print(val_df['label'].value_counts())


Train samples: 7617
Val samples: 1614

Train distribution:
label
Salmonella            2100
Coccidiosis           1981
Healthy               1923
New Castle Disease    1613
Name: count, dtype: int64

Val distribution:
label
Salmonella            525
Coccidiosis           495
Healthy               481
New Castle Disease    113
Name: count, dtype: int64


In [ ]:
# Tạo thư mục cho từng class trong train và val
def organize_data(df, target_dir, images_dir):
    """Tổ chức dữ liệu theo format YOLO (mỗi class một thư mục)"""
    for label in df['label'].unique():
        class_dir = target_dir / label
        class_dir.mkdir(exist_ok=True)

    copied = 0
    for idx, row in df.iterrows():
        img_name = row['images']
        label = row['label']

        src = images_dir / img_name
        dst = target_dir / label / img_name

        if src.exists():
            if not dst.exists():
                shutil.copy2(src, dst)
            copied += 1

        if (idx + 1) % 1000 == 0:
            print(f"  Đã copy: {idx + 1}/{len(df)}...")

    return copied

print("Đang tổ chức train data...")
train_copied = organize_data(train_df, TRAIN_DIR, IMAGES_DIR)
print(f"✓ Đã copy {train_copied} ảnh train")

print("\nĐang tổ chức val data...")
val_copied = organize_data(val_df, VAL_DIR, IMAGES_DIR)
print(f"✓ Đã copy {val_copied} ảnh val")


Đang tổ chức train data...
  Đã copy: 1000/7617...
  Đã copy: 2000/7617...
  Đã copy: 3000/7617...
  Đã copy: 4000/7617...
  Đã copy: 5000/7617...
  Đã copy: 6000/7617...
  Đã copy: 7000/7617...
✓ Đã copy 7617 ảnh train

Đang tổ chức val data...
  Đã copy: 1000/1614...
✓ Đã copy 1614 ảnh val


## Bước 3: Train YOLOv8n-cls


In [ ]:
# Khởi tạo model YOLOv8n-cls (nano - nhỏ nhất, nhanh nhất)
model = YOLO('yolov8n-cls.pt')  # Tự động download pretrained weights

print("Model đã được khởi tạo!")


Model đã được khởi tạo!


In [ ]:
# Train model
results = model.train(
    data=str(YOLO_DATASET),  # Đường dẫn dataset
    epochs=50,                # Số epochs
    imgsz=224,                # Kích thước ảnh (224x224)
    batch=32,                 # Batch size (có thể tăng nếu GPU đủ mạnh)
    device=0,                 # GPU 0 (hoặc 'cpu' nếu không có GPU)
    project='runs/classify',  # Thư mục lưu kết quả
    name='chicken_disease',    # Tên experiment
    exist_ok=True,            # Ghi đè nếu đã tồn tại
    pretrained=True,          # Sử dụng pretrained weights
    optimizer='AdamW',        # Optimizer
    lr0=0.001,               # Learning rate ban đầu
    patience=10,              # Early stopping patience
    save=True,                # Lưu checkpoint
    val=True,                 # Validate trong quá trình train
    plots=True,               # Tạo plots
    verbose=True              # Hiển thị chi tiết
)

print("\n✓ Training hoàn thành!")


Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/AI_Chicken/yolo_dataset, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=chicken_disease, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspe

## Bước 4: Đánh giá kết quả


In [8]:
# Load model tốt nhất
best_model = YOLO('drive/MyDrive/AI_Chicken/runs/classify/chicken_disease/weights/best.pt')

# Validate trên validation set
metrics = best_model.val(data=str(YOLO_DATASET))

print("\nKết quả validation:")
print(f"Top-1 Accuracy: {metrics.top1:.4f}")
print(f"Top-5 Accuracy: {metrics.top5:.4f}")


Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
YOLOv8n-cls summary (fused): 30 layers, 1,440,004 parameters, 0 gradients, 3.3 GFLOPs
train: /content/drive/MyDrive/AI_Chicken/yolo_dataset/train... found 7617 images in 4 classes ✅ 
val: /content/drive/MyDrive/AI_Chicken/yolo_dataset/val... found 1614 images in 4 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 3.7±3.3 ms, read: 4.0±1.6 MB/s, size: 32.3 KB)
val: Scanning /content/drive/MyDrive/AI_Chicken/yolo_dataset/val... 1614 images, 0 corrupt: 100% ━━━━━━━━━━━━ 1614/1614 1.7Mit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 101/101 2.5s/it 4:08
                   all      0.973          1
Speed: 0.0ms preprocess, 13.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/runs/classify/val2

Kết quả validation:
Top-1 Accuracy: 0.9727
Top-5 Accuracy: 1.0000


In [11]:
# Tính Precision, Recall, F1-score cho từng lớp
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Lấy tất cả predictions trên validation set
print("Đang predict trên validation set để tính metrics...")
all_predictions = []
all_true_labels = []

classes_list = sorted(['Coccidiosis', 'Healthy', 'New Castle Disease', 'Salmonella'])

for class_name in classes_list:
    class_dir = VAL_DIR / class_name
    if class_dir.exists():
        img_files = list(class_dir.glob('*.jpg'))
        for img_path in img_files:
            results = best_model.predict(str(img_path), save=False, verbose=False)
            pred_idx = results[0].probs.top1
            pred_label = results[0].names[pred_idx]

            all_true_labels.append(class_name)
            all_predictions.append(pred_label)

# Tính confusion matrix
cm = confusion_matrix(all_true_labels, all_predictions, labels=classes_list)

# Tính Precision, Recall, F1 từ confusion matrix
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, support = precision_recall_fscore_support(
    all_true_labels,
    all_predictions,
    labels=classes_list,
    zero_division=0
)

# Tạo bảng kết quả
print("\n" + "="*80)
print("CHI TIẾT METRICS CHO TỪNG LỚP")
print("="*80)
print(f"{'Lớp':<25} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<10}")
print("-"*80)

for i, class_name in enumerate(classes_list):
    print(f"{class_name:<25} {precision[i]:<12.4f} {recall[i]:<12.4f} {f1[i]:<12.4f} {support[i]:<10}")

# Tính trung bình
macro_precision = np.mean(precision)
macro_recall = np.mean(recall)
macro_f1 = np.mean(f1)
weighted_precision = np.average(precision, weights=support)
weighted_recall = np.average(recall, weights=support)
weighted_f1 = np.average(f1, weights=support)

print("-"*80)
print(f"{'MACRO AVERAGE':<25} {macro_precision:<12.4f} {macro_recall:<12.4f} {macro_f1:<12.4f} {sum(support):<10}")
print(f"{'WEIGHTED AVERAGE':<25} {weighted_precision:<12.4f} {weighted_recall:<12.4f} {weighted_f1:<12.4f} {sum(support):<10}")
print("="*80)

# Hiển thị classification report
print("\n" + "="*80)
print("CLASSIFICATION REPORT (chi tiết)")
print("="*80)
print(classification_report(all_true_labels, all_predictions, labels=classes_list, target_names=classes_list))
print("="*80)


Đang predict trên validation set để tính metrics...

CHI TIẾT METRICS CHO TỪNG LỚP
Lớp                       Precision    Recall       F1-Score     Support   
--------------------------------------------------------------------------------
Coccidiosis               0.9898       0.9758       0.9827       495       
Healthy                   0.9643       0.9543       0.9592       481       
New Castle Disease        0.9912       0.9912       0.9912       113       
Salmonella                0.9609       0.9829       0.9718       525       
--------------------------------------------------------------------------------
MACRO AVERAGE             0.9765       0.9760       0.9762       1614      
WEIGHTED AVERAGE          0.9729       0.9727       0.9727       1614      

CLASSIFICATION REPORT (chi tiết)
                    precision    recall  f1-score   support

       Coccidiosis       0.99      0.98      0.98       495
           Healthy       0.96      0.95      0.96       481
New Cast

In [12]:
# Phân tích Overfitting: So sánh Train vs Validation accuracy

print("="*80)
print("PHÂN TÍCH OVERFITTING")
print("="*80)

# Validate trên train set để so sánh
print("\n1. Đang validate trên TRAIN set...")
train_metrics = best_model.val(data=str(YOLO_DATASET), split='train')

print("\n2. Đang validate trên VAL set...")
val_metrics = best_model.val(data=str(YOLO_DATASET), split='val')

# So sánh
train_acc = train_metrics.top1
val_acc = val_metrics.top1
gap = train_acc - val_acc

print("\n" + "="*80)
print("SO SÁNH TRAIN vs VALIDATION")
print("="*80)
print(f"Train Accuracy:     {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
print(f"Gap (Train - Val):   {gap:.4f} ({gap*100:.2f}%)")
print("="*80)

# Đánh giá
print("\nĐÁNH GIÁ:")
if gap < 0.02:  # < 2%
    print("✓ KHÔNG BỊ OVERFITTING!")
    print("  - Gap rất nhỏ (< 2%)")
    print("  - Model generalize tốt")
elif gap < 0.05:  # < 5%
    print("⚠ OVERFITTING NHẸ")
    print("  - Gap nhỏ (2-5%)")
    print("  - Có thể chấp nhận được")
    print("  - Có thể cải thiện bằng: dropout, data augmentation, regularization")
else:  # >= 5%
    print("⚠ BỊ OVERFITTING!")
    print("  - Gap lớn (>= 5%)")
    print("  - Model học quá kỹ train set")
    print("  - Nên: tăng dropout, tăng augmentation, giảm learning rate")

# Phân tích thêm
print("\n" + "="*80)
print("CÁC DẤU HIỆU KHÁC:")
print("="*80)

# Early stopping
print("✓ Early Stopping đã kích hoạt:")
print("  - Model dừng ở epoch 17 (best)")
print("  - Không train quá lâu → giảm overfitting")

# Metrics từng lớp
print("\n✓ Metrics từng lớp đều tốt:")
print("  - Tất cả lớp đều > 95%")
print("  - Không có lớp nào quá yếu")
print("  - Lớp thiểu số (NCD) performance tốt nhất")

# Kết luận
print("\n" + "="*80)
print("KẾT LUẬN:")
print("="*80)
if gap < 0.02:
    print("Model KHÔNG bị overfitting. Có thể sử dụng ngay!")
else:
    print("Model có dấu hiệu overfitting nhẹ, nhưng vẫn có thể sử dụng.")
    print("Nếu muốn cải thiện, có thể:")
    print("  - Tăng dropout")
    print("  - Tăng data augmentation")
    print("  - Giảm learning rate")
    print("  - Hoặc train ít epochs hơn")
print("="*80)


PHÂN TÍCH OVERFITTING

1. Đang validate trên TRAIN set...
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
train: /content/drive/MyDrive/AI_Chicken/yolo_dataset/train... found 7617 images in 4 classes ✅ 
val: /content/drive/MyDrive/AI_Chicken/yolo_dataset/val... found 1614 images in 4 classes ✅ 
test: None...
train: Fast image access ✅ (ping: 0.7±0.2 ms, read: 0.1±0.0 MB/s, size: 27.9 KB)
train: Scanning /content/drive/MyDrive/AI_Chicken/yolo_dataset/train... 7617 images, 0 corrupt: 100% ━━━━━━━━━━━━ 7617/7617 8.4Mit/s 0.0s
               classes   top1_acc   top5_acc: 100% ━━━━━━━━━━━━ 477/477 4.2s/it 33:01
                   all      0.986          1
Speed: 0.0ms preprocess, 12.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/runs/classify/val3

2. Đang validate trên VAL set...
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
train: /content/drive/MyDrive/AI_Chicken/yolo_dataset/trai